In [51]:
# Tutorial: https://youtu.be/rc_Y6rdBqXM
###!pip install binance
#!pip install python-binance
#!pip install python-decouple

In [52]:
# Binance, including sockets
from binance import BinanceSocketManager
from binance.client import Client
from binance import Client#, ThreadedWebsocketManager, ThreadedDepthCacheManager
import datetime as dt

from secretttfolder import keystestnet

# Standart libraries
import pandas as pd
import numpy as np

import time

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy_utils import database_exists, create_database


# API keys:
api_key = keystestnet.api_key
api_secret = keystestnet.api_secret
client = Client(api_key, api_secret, testnet=True)
client.API_URL = 'https://testnet.binance.vision/api'

# Settings:
symb = "BTCUSDT"
interval = "5MINUTE"

ReadTimeout: HTTPSConnectionPool(host='testnet.binance.vision', port=443): Read timed out. (read timeout=10)

In [ ]:
candles = client.get_klines(symbol=symb, interval=Client.KLINE_INTERVAL_5MINUTE)
numpycandles = np.array(candles)

# reshape date to pandas
df = pd.DataFrame(numpycandles.reshape(-1, 12), dtype=float, columns=('Open Time','Open','High','Low','Close','Volume','Close time','Quote asset volume','Number of trades','Taker buy base asset volume','Taker buy quote asset volume','Ignore'))
df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')

# convert_to_hdf5 = df.to_hdf('prices.h5', key='binance_prices', mode='a')

close_prices = df['Close']


In [ ]:
# Define the RSI indicator
def rsi(price, period=14):
    delta = price.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

# Calculate the RSI indicator
rsi_indicator = rsi(df['Close'])


# Define Divergence 
def divergence(price_set, rsi_window=14, fast_ma = 1, slow_ma = 50):

# Calculate the RSI indicator for using in calculation of divergence 
   rsi_indicator = rsi(price_set)

# Calculate the Divergence indicator. 
   bullish_divergence = ((price_set.diff()<0) & (rsi_indicator.diff() > 0))
   bearish_divergence = ((price_set.diff()<0) & (rsi_indicator.diff() < 0))

   fast_ma_df = df[f'ma{fast_ma}'] = df['Close'].rolling(window=fast_ma).mean()
   slow_ma_df = df[f'ma{slow_ma}'] = df['Close'].rolling(window=slow_ma).mean()
   trend_up = fast_ma_df > slow_ma_df
   trend_down = fast_ma_df < slow_ma_df
   
   # Combine all previous indicators into the one 
   # We need that if bullish_divergence (trend_up), True PLUS fast MA above slow MA, then set 1, everything else - 0. If bearish_divergence True, PLUS fast MA above slow MA (trend_down), then set -1
   indicator = np.where( ( ( (bullish_divergence) == True)  & (trend_up == True) ), 1, 0)
   indicator = np.where( ( ( (bearish_divergence) == True)  & (trend_down == True) ), -1, indicator)
   
   return indicator

diver = divergence(close_prices, rsi_window=14, fast_ma = 1, slow_ma = 50)
last_indicator_value = diver[-1]

In [ ]:
last_indicator_value

0

In [ ]:
last_indicator_value = 1

In [ ]:
last_indicator_value

1

In [ ]:
def trade(ticker, lotsize, open_position = False):
  bm = BinanceSocketManager(client)
  trade_socket = bm.trade_socket(ticker)
  print(dt.datetime.now())
  if last_indicator_value > 0 and last_indicator_value != 0 and not open_position:
    order = client.create_order(symbol=ticker,
                                side='BUY',
                                type='MARKET',
                                quantity=lotsize)
    print(order)
    buyprice = float(order['fills'][0]['price'])
    open_position = True
    return
  if open_position and last_indicator_value < 0 and last_indicator_value != 0:
      order = client.create_order(symbol=ticker,
                                side='SELL',
                                type='MARKET',
                                quantity=lotsize)
      print(order)
      return
  time.sleep(1)  # sleep 1 seconds
  
trade('BTCUSDT', 0.001)

2023-04-27 00:09:35.424656
{'symbol': 'BTCUSDT', 'orderId': 11439939, 'orderListId': -1, 'clientOrderId': 'OGmAhXBDpt56sW0i5RIG9T', 'transactTime': 1682543376264, 'price': '0.00000000', 'origQty': '0.00100000', 'executedQty': '0.00100000', 'cummulativeQuoteQty': '28.27804000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'workingTime': 1682543376264, 'fills': [{'price': '28278.04000000', 'qty': '0.00100000', 'commission': '0.00000000', 'commissionAsset': 'BTC', 'tradeId': 3523714}], 'selfTradePreventionMode': 'NONE'}


In [ ]:
info = client.get_account()
info

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1682543376264,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00100000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9971.71808000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [ ]:
def trade(ticker, lotsize, open_position = False):
  bm = BinanceSocketManager(client)
  trade_socket = bm.trade_socket(ticker)
  print(dt.datetime.now())
  if last_indicator_value > 0 and last_indicator_value != 0 and not open_position:
    order = client.create_order(symbol=ticker,
                                side='BUY',
                                type='MARKET',
                                quantity=lotsize)
    print(order)
    buyprice = float(order['fills'][0]['price'])
    open_position = True
    return
  if open_position and last_indicator_value < 0 and last_indicator_value != 0:
      order = client.create_order(symbol=ticker,
                                side='SELL',
                                type='MARKET',
                                quantity=lotsize)
      print(order)
      return
  time.sleep(1)  # sleep 1 seconds
  
trade('BTCUSDT', 0.0001)

2023-04-27 00:09:37.160219


BinanceAPIException: APIError(code=-1106): Parameter 'timeInForce' sent when not required.

In [ ]:
"""1. Check balance
2. Check if price of current candle is not differ by x% from price from other exchange
3. Check if Ta-Lib indicator value not differ more than y% from another exchange
4. Check open positions
5. Check id position already open
6. Open position characteristics: long/short, timeframe, symbol, indicators values. Are any other positions with same characteristics? If yes, than it's duplicate 
7. Before Closing: check position
8. After closing: position closed? Ok, go check if we have opposite signal. We have opposite signal? Ok, go open opposite Trade Position
9. Position open time
10. How many bars passed
11. Get order ID
"""